In [ ]:
gameFolder = "DOA++-MAME"
#gameFolder = "SFIII-MAME"
#gameFolder = "UMK3-MAME"
#gameFolder = "TEKTAG-MAME"

In [ ]:
import sys, os
import time
import cv2
import tensorflow as tf

timeDepSeed = int((time.time()-int(time.time()-0.5))*1000)

sys.path.append(os.path.join(os.path.abspath(''), '../../games',gameFolder))   

In [ ]:
from makeDiambraEnv import *

In [ ]:
diambraKwargs = {}
diambraKwargs["roms_path"] = "../../roms/MAMEToolkit/roms/"
diambraKwargs["binary_path"] = "../../../customMAME/"
diambraKwargs["player"] = "P1"
diambraKwargs["frame_ratio"] = 3
#diambraKwargs["render"] =True
#diambraKwargs["throttle"] = False
#diambraKwargs["sound"] = False 
#diambraKwargs["character"] ="Random"
diambraKwargs["character"] ="Kasumi"

numEnv=1

wrapperKwargs = {}
wrapperKwargs["frame_stack"] = 1
wrapperKwargs["clip_rewards"] = False
wrapperKwargs["normalize_rewards"] = True
wrapperKwargs["scale"] = True
wrapperKwargs["scale_mod"] = -1
wrapperKwargs["hwc_obs_resize"] = [224, 224, 3]

#keyToAdd = None
keyToAdd = []
keyToAdd.append("actionsBuf")
#keyToAdd.append("player")
keyToAdd.append("healthP1")
keyToAdd.append("healthP2")
keyToAdd.append("positionP1")
keyToAdd.append("positionP2")
#keyToAdd.append("winsP1")
#keyToAdd.append("winsP2")

env = make_diambra_env(diambraMame, env_prefix="Test", num_env=numEnv, seed=timeDepSeed, continue_game = -2,
                       diambra_kwargs = diambraKwargs, wrapper_kwargs = wrapperKwargs,
                       key_to_add = keyToAdd, no_vec = True)

# OR 
#env = make_diambra_env(diambraMame, num_env=numEnv, seed=0, diambra_kwargs = diambraKwargs)
# Frame-stacking with 4 frames
#env = VecFrameStack(env, n_stack=4)

In [ ]:
env.observation_space

In [ ]:
env.observation_space.dtype

In [ ]:
env.observation_space.high

In [ ]:
env.observation_space.low

In [ ]:
observation = env.reset()

shp = observation.shape

additionalPar = int(observation[0,0,shp[2]-1])
print("Additional Par = ", additionalPar)

cumulativeEpRew = 0.0
cumulativeEpRewAll = []
cumulativeTotRew = 0.0

maxNumEp = 100
currNumEp = 0

sess = tf.Session();

while currNumEp < maxNumEp:

    #action = int(input("Action to take"))
    action = env.action_space.sample()
    #action = 11
    #print("Action:", action)
    observation, reward, done, info = env.step(action)
    #print("Observation shape =", observation.shape)
    #input("Pausa")
    #print("Max = ", np.array(observation).astype(np.float32).max())
    #input("Pausa1")
    #print("Observation ch 5 = ", observation[:,:,shp[2]-1])
    #input("Pausa1")
    #addPar = observation[:,:,shp[2]-1]
    #addPar = np.reshape(addPar, (-1))
    #addPar = addPar[1:149]
    #actions = addPar[0:144]
    #actions = np.reshape(actions, (12,-1))
    #print("Actions = ", actions)
    #others = addPar[144:]
    #print("Others = ", others)
    #input("Pausa1")
    #print(np.array(observation).astype(np.float32).shape)
    #input("Pausa2")
    #print(tf.cast(observation, tf.float32).eval(session=sess))
    #doit = True
    #if info["round_done"] == True or doit:
    #    tensor = tf.cast(observation, tf.float32)
    #    tensor2 = tensor[:,:,shp[2]-1]
    #    print("Number off additional param = ", tensor2[0,0].eval(session=sess))    
    #    tensor2 = tf.reshape(tensor2, [-1]) 
    #    #print("After reshaping = ", tensor2)    
    #    tensor2 = tensor2[1:1+additionalPar]
    #    #print("Tensor=", tensor)
    #    tensor_actions = tf.reshape(tensor2[0:additionalPar-4], [12,-1]) 
    #    print("Tensor=", tensor2[additionalPar-4:additionalPar+1].eval(session=sess))
    #    print("Tensor=", tensor_actions.eval(session=sess))
    #    input("Pausa")
        
    #obs = (observation + 1.0) * 0.5 
    #obs = np.array(observation).astype(np.float32)/255.0
        
    #cv2.imshow("image1", obs[:,:,0:3])
    #cv2.imshow("image2", obs[:,:,1])
    #cv2.imshow("image3", obs[:,:,2])
    #cv2.imshow("image4", obs[:,:,3])
    #cv2.waitKey()
    
    #print("Frames shape:", observation.shape)
    #print("Reward:", reward)
    #print("Fighting = ", info["fighting"])
    #print("Rewards = ", info["rewards"])
    #print("HealthP1 = ", info["healthP1"])
    #print("HealthP2 = ", info["healthP2"])
    #print("HealthP1_1 = ", info["healthP1_1"])
    #print("HealthP1_2 = ", info["healthP1_2"])
    #print("HealthP2_1 = ", info["healthP2_1"])
    #print("HealthP2_2 = ", info["healthP2_2"])
    #print("PositionP1 = ", info["positionP1"])
    #print("PositionP2 = ", info["positionP2"])
    #print("WinP1 = ", info["winsP1"]) 
    #print("WinP2 = ", info["winsP2"])

    
    cumulativeEpRew += reward
    
    if np.any(done):
        currNumEp += 1
        print("Ep. # = ", currNumEp)
        print("Ep. Cumulative Rew # = ", cumulativeEpRew)
        sys.stdout.flush()
        cumulativeEpRewAll.append(cumulativeEpRew)
        cumulativeTotRew += cumulativeEpRew
        cumulativeEpRew = 0.0

        observation = env.reset()

print("Mean cumulative reward = ", cumulativeTotRew/maxNumEp)    
print("Mean cumulative reward = ", np.mean(cumulativeEpRewAll))    
print("Std cumulative reward = ", np.std(cumulativeEpRewAll))       
    
env.close()

In [ ]:
# TF tests
aaa = np.zeros((1,3,3))
bbb = np.ones((1,3,3))
aaa = tf.cast(aaa, dtype="float32")
bbb = tf.cast(bbb, dtype="float32")

print(aaa.eval(session=sess))

#ccc = tf.concat( [aaa,bbb], 1)
ccc = tf.layers.flatten(aaa)
print(ccc)
print(ccc.eval(session=sess))
input("AAA")